# Example: The [New York Taxi dataset](https://www.kaggle.com/c/nyc-taxi-trip-duration/data)

This is a large dataset with about 1.5 million data points in 11-dimensional space.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
from models.gpyt_regressor import MyGPModel
from models.gpyt_sparse import TitsiasSparseGP
import torch
import numpy as np
from torch import optim
from gpytorch import likelihoods, mlls

import tqdm

## Data processing

In [55]:
df_train = pd.read_csv("taxi-data/train.csv", index_col="id")

def process_dataframe(df: pd.DataFrame):

    for col in ['pickup_datetime', 'dropoff_datetime']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col])

    df.drop(columns="store_and_fwd_flag", inplace=True)
    df.drop(columns="vendor_id", inplace=True)
    if 'dropoff_datetime' in df.columns:
        df.drop(columns="dropoff_datetime", inplace=True)
    process_datetime_col(df, "pickup_datetime")
    return df
    
def process_datetime_col(df: pd.DataFrame, col, drop=True):
    ser = df[col]
    ser_month = ser.dt.month
    ser_hour = ser.dt.hour
    import re
    date_col = re.sub('datetime', 'month', col)
    df[date_col] = ser_month
    time_col = re.sub('datetime', 'hour', col)
    df[time_col] = ser_hour
    if drop:
        df.drop(columns=col, inplace=True)
    return df

In [56]:
df_train = process_dataframe(df_train)

df_test = pd.read_csv("taxi-data/test.csv", index_col="id")
df_test = process_dataframe(df_test)

The objective function is the RMLSE
$$
    J(s, \hat s) = \sqrt{\sum_i (\log(s_i+1) -\log(\hat s_i + 1)}
$$
we will instead predict $y_i := \log(s_i + 1)$ using the RMSE.

In [64]:
y_train = np.log(1 + df_train.trip_duration.values)
X_train = df_train.drop(columns=['trip_duration']).values

In [65]:
np.random.rand(42)
torch.manual_seed(42)

In [66]:
from sklearn.model_selection import train_test_split

In [67]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
## Convert to Numpy array
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()

### GP regression

In [72]:
import torch.nn.functional as F

In [80]:
l_hood = likelihoods.GaussianLikelihood()

num_induce = 3000
inducing_idx = np.random.choice(np.arange(X_train.shape[0]), size=num_induce,
                                replace=False)
inducing_pts = X_train[inducing_idx]
model = TitsiasSparseGP(X_train, y_train, inducing_pts, l_hood)

marg_hood = mlls.ExactMarginalLogLikelihood(l_hood, model)

In [81]:
def train(model, likelihood, marg_likelihood, optimizer: optim.Optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = -marg_likelihood(output, y_train)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        model.eval()
        y_val_preddist = likelihood(model(X_val))  # predictive distribution of yval | Xtrain,ytrain
        val_loss = F.mse_loss(y_val_preddist.mean, y_val) ** 0.5
    return loss.item(), val_loss.item()

In [82]:
optimizer = optim.Adam(model.parameters())

In [84]:
## DONT RUN THIS!!
## MUST IMPLEMENT BATCHING FIRST!

In [83]:
num_epochs = 500

eprange = tqdm.notebook.trange(num_epochs)
all_losses = []  # marg losses
all_val_losses = []  # all val RMSE
for epoch in eprange:
    loss_, val_loss_ = train(model, l_hood, marg_hood, optimizer)
    all_losses.append(loss_)
    all_val_losses.append(val_loss_)
    eprange.set_postfix({"MLL": loss.item(), "val_rmse": val_score.item()})

KeyboardInterrupt: 

### Inference result

In [ ]:
df_test = pd.read_csv("taxi-data/test.csv")